<a href="https://colab.research.google.com/github/DCDPUAEM/DCDP_2022/blob/main/02-Machine-Learning/notebooks/10-Random-Forest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Random Forest


## El conjunto de datos

Este dataset fue creado por el *National Institute of Diabetes and Digestive and Kidney Diseases* de Estados Unidos. El objetivo del dataset es predecir el diagnostico de cuándo un paciente tiene diabetes o no, basado en ciertas mediciones incluidas en el dataset. Varias restricciones fueron usadas en la selección de estas instancias para filtrar el dataset. En particular, se trata pacientes femeninas de al menos 21 años de edad pertenecientes al grupo indígena Pima de Arizona.

Las variables incluidas son el numero de embarazos la paciente ha tenido, su BMI, nivel de insulina, edad, entre otras.

El dataset se encuentra en https://www.kaggle.com/datasets/uciml/pima-indians-diabetes-database.

In [ ]:
import pandas as pd

url = 'https://github.com/DCDPUAEM/DCDP/raw/main/02-Machine-Learning/data/diabetes.csv'
df = pd.read_csv(url,index_col=0)
df

## Preprocesamiento y Entrenamiento

In [ ]:
# ----- Definimos las features
feature_cols = df.columns.to_list()

X = df[feature_cols].values    # Features
y = df['label'].values         # Target variable

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=7) # 70% training and 30% test

clf = RandomForestClassifier(n_jobs=-1) 

clf = clf.fit(X_train,y_train)  
y_pred = clf.predict(X_test)    # Predecimos usando el conjunto de prueba

¿Cómo lo hizo en el conjunto de prueba?

In [ ]:
from sklearn.metrics import accuracy_score, recall_score, precision_score

y_train_pred = clf.predict(X_train)

print(f"Accuracy: {round(accuracy_score(y_train,y_train_pred),3)}")
print(f"Recall: {round(recall_score(y_train,y_train_pred),3)}")
print(f"Precision: {round(precision_score(y_train,y_train_pred),3)}")

print(f"Accuracy, usando el método score: {clf.score(X_train,y_train)}")

## Resultados

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import seaborn as sns
from sklearn.metrics import accuracy_score, recall_score, precision_score

print(f"Accuracy: {round(accuracy_score(y_test,y_pred),3)}")
print(f"Recall: {round(recall_score(y_test,y_pred),5)}")
print(f"Precision: {round(precision_score(y_test,y_pred),3)}")

target_labels = ['no diabetes','diabetes']

plt.figure(figsize=(3,3))
cm = confusion_matrix(y_test,y_pred)
s_cm = sns.heatmap(cm,cmap='plasma',annot=True, fmt='g',
            xticklabels=target_labels,
            yticklabels=target_labels)
s_cm.set(xlabel='Predicted',ylabel='Real')
plt.show()

Podemos obtener la importancia de las features de acuerdo a la reducción de impureza de los nodos donde participan. 

In [ ]:
import numpy as np

importances = pd.DataFrame({'feature':feature_cols,'importancia':np.round(clf.feature_importances_,3)})
importances.sort_values(by='importancia',ascending=False,inplace=True)
importances.set_index('feature')

## Exploración del random forest

Podemos explorar el conjunto de árboles de decisión construidos por el random forest mediante la lista `estimators_`. Cada elemento de esta lista es un árbol de decisión como los que usamos la sesión pasada.

¿Cuántos árboles se usaron?

In [ ]:
print(f"Número de árboles: {len(clf.estimators_)}")

Analicemos la predicción para un elemento arbitrario de X_test

In [ ]:
import numpy as np

idx0 = 37
new_x = X_test[idx0]
print(f"Etiqueta real: {y_test[idx0]}, Preddición: {y_pred[idx0]}")

predicciones = []
for dt in clf.estimators_:
    predicted_label = int(dt.predict([X_test[idx0]])[0])  # No podemos pasar sólo un renglón de X_test, tiene que ser una matriz, por lo que pasamos una matriz de tamaño 1 x num_features
    predicciones.append(predicted_label)

# #Usando list comprehension:
# predicciones = [int(dt.predict([X_test[idx0]])[0]) for dt in clf.estimators_] 

print(f"Las primeras 10 predicciones: {predicciones[:10]}")

# ---- Contamos cuántos votos tuvo cada etiqueta por parte del bosque ----

zeros = np.where(np.array(predicciones)==0)[0]
ones = np.where(np.array(predicciones)==1)[0]

print(f"{zeros.shape[0]} árboles que predijeron la etiqueta 0:\n{zeros}\n")
print(f"{ones.shape[0]} Árboles que predijeron la etiqueta 1:\n{ones}\n")

Exploremos un árbol arbitrario del ensamble. 

In [ ]:
from sklearn import tree

dt = clf.estimators_[1]

print(f"Profundidad del árbol: {dt.get_depth()}")
print(f"Número de hojas del árbol: {dt.get_n_leaves()}")

text_representation = tree.export_text(decision_tree=dt,
                                    feature_names=feature_cols)
# print(text_representation)

Exploremos estadísticamente todos los árboles individuales del ensamble

In [ ]:
import seaborn as sns

profundidades = [dt.get_depth() for dt in clf.estimators_]
hojas = [dt.get_n_leaves() for dt in clf.estimators_]

fig, axs = plt.subplots(1,2,figsize=(9,5),sharey=True)
axs[0].set_title("Historgrama de profundidades")
sns.histplot(profundidades,ax=axs[0])
axs[1].set_title("Historgrama de hojas")
sns.histplot(hojas,ax=axs[1])
fig.show()


## Comparación con el DecisionTreeClassifier

⭕ **Práctica**:

1. Encuentra el árbol del ensamble con el mejor rendimiento, respecto al accuracy, 
2. Vamos a compararlo con el mejor árbol de decisión de la práctica pasada: 
    * Compara la profundidad y número de hojas de ambos.
    * Compara los rendimientos de ambos-
    * ¿Puedes compararlos en las otras métricas de rendimiento? Precision, Recall, F1-score.